# The Sims-Flanagan gradients
This notebook is the result of a few days of blood shed with manual derivatives assembly. 
It benchmarks a python version of the computation of a sims-flanagan forward only leg and the same done using heyoka.

In [2]:
import heyoka as hy
import numpy as np
import pykep as pk

from numba import njit

## Problem data

In [37]:
# Problem data
mu = pk.MU_SUN
max_thrust = np.random.random()
isp = 3000.0*np.random.random()
veff = isp * 9.80665

# Initial state
ms = 1500.0 * np.random.random()
rs = np.array([1, 0.1, -0.1]) * pk.AU
vs = np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY

# Variables
nseg = 5
tof = 324.0 * pk.DAY2SEC * (np.random.random() + 1)
throttles = (0.5-np.random.random(3*nseg))

# C++ test
ms = 1500.0
rs = np.array([1, 0.1, -0.1]) * pk.AU
vs = np.array([0.2, 1, -0.2]) * pk.EARTH_VELOCITY
throttles = np.array([0.01] * 3 * nseg)
tof = 324.0 * pk.DAY2SEC
max_thrust = 0.01
isp = 3000
veff = isp * 9.80665


In [38]:
tof

27993600.0

## The forward computation

In [35]:
# The dynamics
def dyn(r, v, mu):
    R3 = np.linalg.norm(r) ** 3
    f = [v[0], v[1], v[2], -mu / R3 * r[0], -mu / R3 * r[1], -mu / R3 * r[2]]
    return np.array(f).reshape(-1, 1)

In [36]:
# This is the main computation to be ported in C++. Manual assembly of the gradients.
xs, ys, zs, vxs, vys, vzs, ms, throttles, tof = rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles, tof
nseg = int(throttles.shape[0] / 3)
c = max_thrust * tof / nseg
a = 1. / veff
dt = tof / nseg
m0=ms
# We define the indepedent variables
u = []
du = []
m = [m0]
for i in range(nseg):
    u.append(np.array(throttles[i*3:i*3+3]).reshape(1,3))
    du.append(np.zeros((3, 3*nseg+2)))
    du[i][:, 3*i:3*i+3] = np.eye(3)
dm = [np.hstack((np.zeros((1, 3*nseg)), np.eye(1), np.zeros((1, 1))))]
dtof = np.hstack((np.zeros((1, 3*nseg+1)), np.eye(1)))
Dv = []
dDv = []

# 1 - We compute the mass schedule and related quantities
for i in range(nseg):
    Dv.append(c / m[i] * u[i])
    un = np.sqrt(u[i][0][0]**2+u[i][0][1]**2+u[i][0][2]**2)
    Dvn = c / m[i] * un
    dDv.append(c / m[i] * du[i] - c / m[i]**2 * u[i].T@dm[i] + max_thrust / m[i] * u[i].T@dtof / nseg)
    dDvn = c / m[i] / un * u[i] @ du[i] - c / m[i]**2 * un * dm[i] + max_thrust / m[i] * un * dtof / nseg
    m.append(m[i] * np.exp(-Dvn * a))
    dm.append(-m[i+1] * a * dDvn + np.exp(-Dvn * a) * dm[i])

M=[]
f=[]
rv_it = [[xs, ys,zs],[vxs,vys,vzs]]
tofs = [dt]*(nseg+1)
tofs[0]/=2.
tofs[-1]/=2.

# 2 - We compute the state transition matrices
for i in range(nseg+1):
    rv_it, M_it = pk.propagate_lagrangian(
        rv=rv_it, tof=tofs[i], mu=mu, stm=True)
    M.append(M_it)
    #We compute f
    r = rv_it[0]
    f.append(dyn(r, rv_it[1], mu))
    #And add the impulse if needed
    if i<nseg:
        rv_it[1] = [a + b for a, b in zip(rv_it[1], Dv[i].flatten())]


# 3 - We assemble the gradients
Iv = np.diag((0, 0, 0, 1, 1, 1))[:, 3:]
# Mc will contain [Mn@..@M0,Mn@..@M1, Mn]
Mc=[0]*(nseg+1)
Mc[-1] = M[-1]
for i in range(1, len(M)):
    Mc[-1-i] = Mc[-1-i+1]@M[-1-i]
# grad_tof 
grad_tof = 0.5 * f[-1]
for i in range(nseg-1):
    grad_tof+= Mc[i+2]@f[i+1]
grad_tof+= 0.5 * Mc[1]@f[0]
grad_tof/=nseg
for i in range(nseg):
    grad_tof+=Mc[i+1]@Iv@dDv[i][:, -1:]

# grad u
grad_u=0
for i in range(nseg):
    grad_u += Mc[i+1]@Iv@dDv[i][:, :-2]

# grad ms
grad_ms=0
for i in range(nseg):
    grad_ms += Mc[i+1]@Iv@dDv[i][:, -2:-1]

# grad xs
grad_xs = Mc[0]

# Assembling te return value
# grad will contain the gradient of the final posvelm with respect to the throttles and the tof
grad = np.hstack((grad_u, grad_tof))
grad = np.vstack((grad, dm[-1][:,:-1]))
grad[-1,-1] = dm[-1][:,-1][0]
# grad_ic will contain the gradient of the final posvelm with respect to the initial posvelm
grad_ic = np.vstack((grad_xs, np.zeros((1,6))))
grad_ic = np.hstack((grad_ic, np.zeros((7,1))))
grad_ic[:6,-1:] = grad_ms
grad_ic[-1,-1] = dm[-1][:,-2][0]


In [6]:
len(m)

6

In [7]:
# This is the main computation to be ported in C++. Manual assembly of the gradients.
def manual(xs, ys, zs, vxs, vys, vzs, ms, throttles, tof): 
    nseg = int(throttles.shape[0] / 3)
    c = max_thrust * tof / nseg
    a = 1. / veff
    dt = tof / nseg
    m0=ms
    # We define the indepedent variables
    u = []
    un = []
    du = []
    m = [m0]
    for i in range(nseg):
        u.append(np.array(throttles[i*3:i*3+3]).reshape(1,3))
        du.append(np.zeros((3, 3*nseg+2)))
        du[i][:, 3*i:3*i+3] = np.eye(3)
    dm = [np.hstack((np.zeros((1, 3*nseg)), np.eye(1), np.zeros((1, 1))))]
    dtof = np.hstack((np.zeros((1, 3*nseg+1)), np.eye(1)))
    Dv = []
    dDv = []

    # 1 - We compute the mass schedule and related quantities
    for i in range(nseg):
        Dv.append(c / m[i] * u[i])
        un = np.sqrt(u[i][0][0]**2+u[i][0][1]**2+u[i][0][2]**2)
        Dvn = c / m[i] * un
        dDv.append(c / m[i] * du[i] - c / m[i]**2 * u[i].T@dm[i] + max_thrust / m[i] * u[i].T@dtof / nseg)
        dDvn = c / m[i] / un * u[i] @ du[i] - c / m[i]**2 * un * dm[i] + max_thrust / m[i] * un * dtof / nseg
        m.append(m[i] * np.exp(-Dvn * a))
        dm.append(-m[i+1] * a * dDvn + np.exp(-Dvn * a) * dm[i])

    M=[]
    f=[]
    rv_it = [[xs, ys,zs],[vxs,vys,vzs]]
    tofs = [dt]*(nseg+1)
    tofs[0]/=2.
    tofs[-1]/=2.

    # 2 - We compute the state transition matrices
    for i in range(nseg+1):
        rv_it, M_it = pk.propagate_lagrangian(
            rv=rv_it, tof=tofs[i], mu=mu, stm=True)
        M.append(M_it)
        #We compute f
        r = rv_it[0]
        f.append(dyn(r, rv_it[1], mu))
        #And add the impulse if needed
        if i<nseg:
            rv_it[1] = [a + b for a, b in zip(rv_it[1], Dv[i].flatten())]


    # 3 - We assemble the gradients
    Iv = np.diag((0, 0, 0, 1, 1, 1))[:, 3:]
    # Mc will contain [Mn@..@M0,Mn@..@M1, Mn]
    Mc=[0]*(nseg+1)
    Mc[-1] = M[-1]
    for i in range(1, len(M)):
        Mc[-1-i] = Mc[-1-i+1]@M[-1-i]
    # grad_tof 
    grad_tof = 0.5 * f[-1]
    for i in range(nseg-1):
        grad_tof+= Mc[i+2]@f[i+1]
    grad_tof+= 0.5 * Mc[1]@f[0]
    grad_tof/=nseg
    for i in range(nseg):
        grad_tof+=Mc[i+1]@Iv@dDv[i][:, -1:]

    # grad u
    grad_u=0
    for i in range(nseg):
        grad_u += Mc[i+1]@Iv@dDv[i][:, :-2]

    # grad ms
    grad_ms=0
    for i in range(nseg):
        grad_ms += Mc[i+1]@Iv@dDv[i][:, -2:-1]

    # grad xs
    grad_xs = Mc[0]

    # Assembling te return value
    # grad will contain the gradient of the final posvelm with respect to the throttles and the tof
    grad = np.hstack((grad_u, grad_tof))
    grad = np.vstack((grad, dm[-1][:,:-1]))
    grad[-1,-1] = dm[-1][:,-1][0]
    # grad_ic will contain the gradient of the final posvelm with respect to the initial posvelm
    grad_ic = np.vstack((grad_xs, np.zeros((1,6))))
    grad_ic = np.hstack((grad_ic, np.zeros((7,1))))
    grad_ic[:6,-1:] = grad_ms
    grad_ic[-1,-1] = dm[-1][:,-2][0]
    return (grad, grad_ic)

In [8]:
%%timeit
grad, grad_ic = manual(rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles, tof)

663 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
grad, grad_ic = manual(rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms, throttles, tof)

# Test with heyoka
We compile the same function bu using heyoka

In [10]:
def propagate_lagrangian_heyoka(pos_0, vel_0, mu, tof):
    x0, y0, z0 = pos_0
    vx0, vy0, vz0 = vel_0
    v02 = vx0**2 + vy0**2 + vz0**2
    r0 = hy.sqrt(x0**2 + y0**2 + z0**2)
    eps = v02 * 0.5 - mu / r0
    a = -mu / (2.0 * eps)

    sigma0 = np.dot(pos_0, vel_0) / np.sqrt(mu)
    s0 = sigma0 / hy.sqrt(a)
    c0 = 1.0 - r0 / a

    n = hy.sqrt(mu / (a * a * a))
    DM = n * tof

    DE = hy.kepDE(s0, c0, DM)

    # Compute cos(DE) and sin(DE).
    cDE = hy.cos(DE)
    sDE = hy.sin(DE)

    r = a + (r0 - a) * cDE + sigma0 * hy.sqrt(a) * sDE

    F = 1.0 - a / r0 * (1.0 - cDE)
    G = a * sigma0 / np.sqrt(mu) * (1.0 - cDE) + r0 * hy.sqrt(a / mu) * sDE
    Ft = -hy.sqrt(mu * a) / (r * r0) * sDE
    Gt = 1 - a / r * (1.0 - cDE)

    pos = F * pos_0 + G * vel_0
    vel = Ft * pos_0 + Gt * vel_0

    return [pos, vel]

def ground_truth(xs, ys, zs, vxs, vys, vzs, ms, throttles_vars, tof):
    nseg = int(len(throttles) / 3)
    dt = tof / nseg
    c = max_thrust * dt
    
    pos = np.array([xs, ys, zs])
    vel = np.array([vxs, vys, vzs])
    m=ms

    for i in range(nseg):
        ux, uy, uz = throttles_vars[3*i:3*i+3]
        if i == 0:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
        else:
            pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt)  
        vel = vel + np.array([ux, uy, uz]) * c / m
        m = m * hy.exp(-hy.sqrt(ux**2+uy**2+uz**2) * c / veff / m)
    # 4
    pos, vel = propagate_lagrangian_heyoka(pos, vel, mu, dt/2)
    retval = np.hstack([pos, vel, m])
    return retval

In [11]:
# The symbolic variables
xs_var, ys_var, zs_var = hy.make_vars("xs", "ys", "zs")
vxs_var, vys_var, vzs_var = hy.make_vars("vxs", "vys", "vzs")
ms_var, = hy.make_vars("ms")
tof_var, = hy.make_vars("t")
throttles_symbols = []
for i in range(nseg):
    throttles_symbols.extend(["ux"+str(i), "uy"+str(i), "uz"+str(i)])
throttles_vars = hy.make_vars(*throttles_symbols)

In [12]:
# Here we compile the two functions for the two gradients
xf = ground_truth(xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, throttles_vars, tof_var)

dtens_ic = hy.diff_tensors(xf,
                    diff_args=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var],
                    diff_order=1
                    )
dtens = hy.diff_tensors(xf,
                    diff_args=[*throttles_vars, tof_var],
                    diff_order=1
                    )
jac_ic = dtens_ic.jacobian   
jac = dtens.jacobian   

d_cf = hy.make_cfunc(jac.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])

d_cf_ic = hy.make_cfunc(jac_ic.flatten(),
                       # Specify the order in which the input
                       # variables are passed to the compiled
                       # function.
                       vars=[xs_var, ys_var, zs_var, vxs_var, vys_var, vzs_var, ms_var, *throttles_vars, tof_var])


In [13]:
%%timeit
analytical_grad_ic = d_cf_ic([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])
analytical_grad = d_cf([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])

18.2 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
analytical_grad_ic = d_cf_ic([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])
analytical_grad = d_cf([rs[0], rs[1], rs[2], vs[0], vs[1], vs[2], ms] + throttles.tolist() + [tof])

In [15]:
## Lets check the difference in he values computed is zero / small

In [16]:
(analytical_grad.reshape(7,-1)-grad)/grad

array([[            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             

In [17]:
(analytical_grad_ic.reshape(7,-1)-grad_ic)/grad_ic

/tmp/ipykernel_2060112/645163605.py:1: RuntimeWarning: invalid value encountered in divide
  (analytical_grad_ic.reshape(7,-1)-grad_ic)/grad_ic


array([[            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
                    nan],
       [            nan,             nan,             nan,
                    nan,             nan,             nan,
        -7.2404375